#### Document Intelligence REST API 실습

In [2]:
#%pip install gradio

In [ ]:
import requests
import time
import random
from PIL import ImageFont
import platform
import os
from dotenv import load_dotenv
# .env 환경변수 로드
load_dotenv()

# ---------------------------------------------------------
# 2. 유틸리티 함수들 (색상, 폰트)
# ---------------------------------------------------------
def random_color():
    """
    시각화를 위해 랜덤한 RGB 색상 튜플 (R, G, B)을 반환합니다.
    """
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 

def get_font():
    """
    운영체제(OS)에 따라 한글이 지원되는 적절한 폰트 객체를 반환합니다.
    이미지에 텍스트를 그릴 때 깨짐을 방지하기 위함입니다.
    """
    font_size = 10
    
    try:
        if platform.system() == "Windows":
            # 윈도우: 맑은 고딕
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            # 맥: 애플 고딕
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux 등
            # 기본 폰트 (한글 지원 안 될 수 있음)
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 지정한 폰트 파일이 없을 경우 PIL 기본 폰트 사용
        return ImageFont.load_default()
    
# 문서 인텔리전스 REST API 요청
def request_document_intelligence(image_path) :

    ## 1. EndPoint
    endPoint = "https://8ai022-document.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-read:analyze?_overload=analyzeDocument&api-version=2024-11-30"

    ## 2. Method - POST 

    ## 3. Header
    headers = {
        "Content-Type" : "image/*",
        "Ocp-Apim-Subscription-Key" : os.getenv("OCP_APIM_SUB_KEY")
    }

    ## 4. Body
    # with open : 자동 close 하여, 자원 반환함
    with open(image_path, "rb") as image_file : 
        image_data = image_file.read()
        

    body = image_data

    # (1) 분석 요청 (비동기인경우, 작업이 완료되지 않아도 202를 내려주기 때문에 아래에서 1초마다 GET 요청을 보냄)
    response = requests.post(endPoint, headers=headers, data=body)
    
    if response.status_code != 202 : 
        print("Error : ", response.status_code, response.text)
        return None 
    
    # (2) 분석 결과 요청 (상태가 succeded일때까지)
    url = response.headers["Operation-Location"]

    while True : 
        result_response = requests.get(url, headers=headers)

        # get 정상 통신
        if result_response.status_code != 200 :
            print("Error : ", response.status_code, response.text)
            return None
        
        result_json = result_response.json()
        current_status = result_json.get("status")
        
        # 상태가 succeeded일 때까지, 반복 GET 요청
        if current_status == "succeeded" : 
            import json
            return json.loads(result_response.text) # json 문자열 dict로 변환
        elif current_status == "running" : 
            print(f'http status is : {current_status}...')
            time.sleep(1)
        elif current_status in ["failed", "canceled"]:
            # 실패 또는 취소 시 오류 출력 후 종료
            print(f"Analysis failed or was canceled. Details: {current_status}")
            return None
        else:
            # 알 수 없는 상태
            print("Unknown status received:  Stopping polling.", current_status)
            return None
        
# 이미지에, response_data를 이용해 바운딩 박스 그리기
def draw_image(image_path, response_data) :
    # 파이썬 이미지 라이브러리
    from PIL import Image, ImageDraw

    image = Image.open(image_path) # 이미지 PIL 형태로 변환
    draw = ImageDraw.Draw(image)   # PIL 그릴 수 있는 형태로 변환
    
    block_list = response_data.get("analyzeResult").get("paragraphs")

    # 이미지에, 빨간색으로 영역 그리기
    for block in block_list : 
        color = random_color()
        content = block.get("content")

        # 이미지 좌상단 좌표를 기준으로 한 X, Y 좌표들의 플랫 리스트
        # API는 [x1, y1, x2, y2, ...] 형태의 리스트로 반환합니다.
        polygon = block.get("boundingRegions")[0].get("polygon")

        # 그리기 편하도록 (x, y) 튜플의 리스트로 변환합니다.
        # 예: [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
        pairs = list(zip(polygon[::2], polygon[1::2]))

        # 1. 다각형(영역) 그리기
        draw.polygon(pairs, outline=color)

        # 2. 텍스트 그리기 (첫 번째 좌표보다 20픽셀 위에 텍스트 표시)
        draw.text((pairs[0][0], pairs[0][1] - 20), content, fill=color, font=get_font())

    return image



import gradio as gr


with gr.Blocks(gr.themes.Soft()) as demo : 
    gr.Markdown("Document Intelligence")

    # 전달받은 이미지 그대로 리턴
    # 지역에서만 사용할 목적의 함수
    def change_image(image_path) :
        # Document Intelligence API 호출하여, JSON 형식 응답
        response_data = request_document_intelligence(image_path)

        # 이미지 경로와, Document Intelligence API 호출하여, JSON 형식 응답 전달하여 이미지에 바운딩 박스 그리기
        image = draw_image(image_path, response_data)
        return image

    with gr.Row() : 
        input_image = gr.Image(label="문서 이미지", type="filepath")
        output_image = gr.Image(label="결과 이미지", type="pil")
    
    # 좌측 이미지 변경(업로드) 시, change_image 핸들러 동작하여, 결과를 output_image에 전달
    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])

# share=True 공개 도메인 생성
demo.launch(share=True)
#change_image(r"files/test4.png")




* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://cd207e4927be9f2c6c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastapi\applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\sit